In [147]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import urllib.request
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [165]:
import requests
import io

#Create the DB, to run use
#sudo docker run -p 9000:9000  -p 9009:9009  -p 8812:8812  -p 9003:9003  questdb/questdb

q = 'create table forecasts'\
    '(location string,'\
    'Date string,'\
    'snowfall float,'\
    'maxTemp float,'\
    'maxWind float,'\
    'sun_percent float)'
r = requests.get("http://localhost:9000/exec?query=" + q)
print(r.status_code)

200


In [166]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_driver = os.path.join(os.getcwd(), "chromedriver")
driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver)

In [189]:
class location:
    def __init__(self,name,lat,long):
        self.name=name
        self.lat=lat
        self.long=long
    
    def get_forecast(self):
        url="https://spotwx.com/products/grib_index.php?model=gem_lam_continental&lat="+str(self.lat)+"&lon="+str(self.long)+"&tz=America/Vancouver&display=table"
        driver.get(url)
        driver.execute_script("$('.buttons-csv').click();")
        time.sleep(2)
        df=pd.read_csv("SpotWx.csv")
        os.remove("SpotWx.csv")
        self.forecast=df
        return df
    
    def weather_synopsis(self,df):
        new_snow=df["SQP"].iloc[-1] #get last accumulated snow
        melting=df["TMP"].max()>0
        wind=df["WS"].max()
        #sun during the day hours only
        cloud=df.CLOUD.mask((df.TIME.str.slice(stop=2).astype(int)<8) | (df.TIME.str.slice(stop=2).astype(int)>20))
        sun=100-cloud.mean()
        data = {'New Snow':  [new_snow],
                'Melting': [melting],
                'Max Winds': [wind],
                'Weighted Sun':[sun]
               }
        result = pd.DataFrame(data, index=[self.name])
        return result
    
    def store_today(self,df):
        #Check to see if we already have the data
        #r = requests.get("http://localhost:9000/exec?query=select count() from forecasts where date="+today.DATE+" and location="+self.name)
        #if ("r.text"!="0"):
        today=df.iloc[0:df[df.TIME == '00:00'].index[0]]
        cloud=today.CLOUD.mask((today.TIME.str.slice(stop=2).astype(int)<8) | (today.TIME.str.slice(stop=2).astype(int)>20))
        sun=100-cloud.mean()
        date=today.DATE[0].replace("/","-")
        query = "insert into forecasts values('"\
        + self.name + "','"\
        + date + "',"\
        + str(today["SQP"].iloc[-1]) + ","\
        + str(today.TMP.max()) + ","\
        + str(today.WS.max()) + ","\
        + str(sun) + ")"
        r = requests.get("http://localhost:9000/exec?query=" + query)
        
        if (r.status_code==200):
            return True
        else:
            return False

IndentationError: expected an indented block (1987508578.py, line 36)

In [180]:
blackcomb=location("black",50.09970, -122.86943)

garibaldi=location("gar",49.93299, -123.03600)

joffre_lake=location("jof",50.34143, -122.47768)

s2s=location("s2s",9.65218, -123.09211)

cloudburst=location("cloud",49.93298, -123.23602)

manning=location("man",49.06805, -120.91646)

coq=location("coq",49.61094, -121.05758)

rogers=location("rog",51.30123, -117.52014)

sites=[blackcomb,garibaldi,joffre_lake,cloudburst,manning,coq,rogers]

In [181]:
view=pd.DataFrame(columns = ['New Snow', 'Melting','Max Winds','Weighted Sun'])

In [182]:
print("Grabbing site data...")
for site in sites:
    forecast=site.get_forecast()
    if(site.store_today(forecast)):
        print(site.name)
    result=site.weather_synopsis(forecast)
    view=view.append(result)

Grabbing site data...
http://localhost:9000/exec?query=insert into forecasts values('black','2022-02-15',0.0,-3.0,5,56.2)
black
http://localhost:9000/exec?query=insert into forecasts values('gar','2022-02-15',0.1,-1.5,8,23.400000000000006)
gar
http://localhost:9000/exec?query=insert into forecasts values('jof','2022-02-15',0.0,-2.3,3,31.400000000000006)
jof
http://localhost:9000/exec?query=insert into forecasts values('cloud','2022-02-15',0.1,2.2,6,20.0)
cloud
http://localhost:9000/exec?query=insert into forecasts values('man','2022-02-15',0.1,-2.5,11,20.0)
man
http://localhost:9000/exec?query=insert into forecasts values('coq','2022-02-15',1.4,-1.8,14,20.0)
coq
http://localhost:9000/exec?query=insert into forecasts values('rog','2022-02-15',2.7,-6.8,10,24.799999999999997)
rog


In [ ]:
view
view.style.background_gradient(cmap="RdYlGn",subset=["New Snow"]) #style snow column

In [228]:
view.style.background_gradient(cmap="cividis",subset=["Weighted Sun"],vmin=50) #style sun column

,New Snow,Melting,Max Winds,Weighted Sun
black,0.000000,True,4,100.000000
gar,0.000000,True,12,100.000000
jof,0.000000,True,8,100.000000
cloud,0.000000,True,4,98.703704
man,0.000000,True,9,97.333333
coq,0.000000,True,10,94.296296
rog,0.000000,False,11,98.888889


In [190]:
driver.quit()